In [70]:
import math

import ipywidgets as widgets
import jupyter_cadquery as jcq
from jupyter_cadquery.replay import replay, show

from build123d import *

In [71]:
jcq.set_defaults(
    control="orbit",
    axes=True,
    axes0=True,
    grid=(True, False, False),
    glass=True,
    theme="browser",
)

# open_viewer("Cadquery")
replay_box = widgets.Checkbox(False, description="Enable Replay")
out = widgets.Output()
display(replay_box)
display(out)


@out.capture()
def changed(e):
    if e["new"]:
        jcq.replay.enable_replay(warning=False)
    else:
        jcq.replay.disable_replay()


replay_box.observe(changed, names="value")

Checkbox(value=False, description='Enable Replay')

Output()

In [72]:
layer_height = 0.2
handle_height = 15
handle_length = 40
reinforcement_length = 30
thickness = 1.5

In [73]:
with BuildLine() as side:
    bottom = Line((0, 0), (handle_length, 0))
    top = Line(bottom @ 0 + (10, handle_height), bottom @ 1 + (0, handle_height))
    c = Line(bottom @ 0, top @ 0)
    Offset(bottom, c, top, amount=thickness / 2, kind=Kind.INTERSECTION)

show(side.line)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [76]:
with BuildSketch() as side_sk:
    with Locations((0, thickness / 2)):
        Add(side.line)
        MakeFace()
    Rectangle(
        handle_length,
        handle_height + thickness,
        align=(Align.MIN, Align.MIN),
        mode=Mode.INTERSECT,
    )
    with BuildLine() as rein_line:
        start_pts = side_sk.vertices() << Axis.X
        end_point_top = (-reinforcement_length, layer_height)
        top_line = Polyline(
            (start_pts >> Axis.Y).first.to_vector(),
            (-reinforcement_length / 10, thickness),
            (-reinforcement_length / 3, max(thickness / 2, layer_height)),
            end_point_top,
        )
        end_point_bottom = (-reinforcement_length, 0)
        bottom_line = Line((start_pts << Axis.Y).first.to_vector(), end_point_bottom)
        Line(top_line @ 1, bottom_line @ 1)
    MakeFace()

show(side_sk.sketch)

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…

In [77]:
with BuildPart() as out:
    total_length = reinforcement_length + handle_length
    with Locations((reinforcement_length, 0)):
        Add(side_sk.sketch)
    Extrude(amount=total_length, both=True)
    Box(
        total_length,
        handle_height,
        thickness,
        align=(Align.MIN, Align.MIN, Align.CENTER),
    )
    with Workplanes(Plane.XZ):
        with BuildSketch() as mask:
            RegularPolygon(total_length, side_count=4)
            RegularPolygon(
                total_length - 10 / math.sin(math.radians(45)),
                side_count=4,
                mode=Mode.SUBTRACT,
            )
        Extrude(amount=-(handle_height + thickness), mode=Mode.INTERSECT)

show(out.part)
out.part.export_step("/tmp/x.step")

CadViewerWidget(anchor=None, cad_width=800, glass=True, height=600, pinning=False, theme='browser', title=None…


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : /tmp/x.step(1694 ents)  Write  Done


<IFSelect_ReturnStatus.IFSelect_RetDone: 1>